In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey
from haversine import haversine
from pprint import pprint

# needed to clear output below
from IPython.display import clear_output

In [3]:
fire_data = pd.read_csv("Fire.csv")
fire_data.head(1)

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000.0,1.01,1.23


In [4]:
dallas_coords_df = pd.read_csv("data/Dallas_coords_grid.csv")
dallas_coords_df.head(1)

,Lat,Lng,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,33.01025,-96.481611,2.21,2.76


In [9]:
fire_stations_df = pd.read_csv("data/fire_stations.csv")
fire_stations_df.head()

,Address,Lat,Lng
0,1901 Irving Blvd.,32.794005,-96.827024
1,4211 Northhaven Road,32.902599,-96.837838
2,500 N. Malcolm X Blvd.,32.787163,-96.783337
3,816 South Akard Street,32.774245,-96.796156
4,2039 St. Augustine Drive,32.675624,-96.657310


### Store fire station coords as list of lat/long

In [40]:
station_coords = []
station_coords = [[row['Lat'], row['Lng']] for index, row in fire_stations_df.iterrows()]
station_coords

[[32.7940047, -96.8270241],
 [32.9025987, -96.8378375],
 [32.7871627, -96.7833368],
 [32.7742453, -96.7961557],
 [32.6756243, -96.6573101],
 [32.7636404, -96.7721515],
 [32.9758442, -96.803919],
 [32.8101807, -96.776478],
 [32.67886860000001, -96.6101274],
 [32.9980252, -96.8323265],
 [32.8101255, -96.80968620000002],
 [32.6423703, -96.9530806],
 [32.9972654, -96.784076],
 [32.7416119, -96.8397527],
 [32.7486012, -96.8224998],
 [32.7754857, -96.9042909],
 [32.8176289, -96.7619042],
 [32.7836921, -96.8035866],
 [32.793779, -96.7468528],
 [32.9222122, -96.8117456],
 [32.8508639, -96.8592024],
 [32.9152172, -96.7687762],
 [32.72600560000001, -96.798973],
 [32.7521418, -96.7573146],
 [32.6840284, -96.7875083],
 [32.7445409, -96.8746074],
 [32.8652539, -96.8088819],
 [32.8955844, -96.7523766],
 [32.9163345, -96.7170817],
 [32.8946709, -96.8833143],
 [32.8326359, -96.7043139],
 [32.7743973, -96.6997596],
 [32.7196172, -96.8365236],
 [32.7337305, -96.6912531],
 [32.8809282, -96.8517692],
 [32

### Store distance to two nearest fire stations from each fire event.

In [24]:
station_dist_1 = []
station_dist_2 = []

for index, row in fire_data.iterrows():
    

    lat = row['Lat']
    lng = row['Lng']
    dist_list = []

    for index1, row1 in fire_stations_df.iterrows():
        lat1 = row1['Lat']
        lng1 = row1['Lng']
        dist = haversine(lat1,lng1,lat,lng,'mi')
        dist_list.append(dist)
    dist_list.sort()
    station_dist_1.append(dist_list[0])
    station_dist_2.append(dist_list[1])

fire_data['Dist to 1st Station (mi)'] = station_dist_1
fire_data['Dist to 2nd Station (mi)'] = station_dist_2
fire_data['Dist to 1st Station (mi)'] = fire_data['Dist to 1st Station (mi)'].round(2)
fire_data['Dist to 2nd Station (mi)'] = fire_data['Dist to 2nd Station (mi)'].round(2)

try:
    fire_data.to_csv('Fire.csv',index=False)
    print('save successful')
except:
    print('save fail')


save successful


### Distance to nearest station for entire city of Dallas, regardless of fire event.

In [25]:
station_dist_1 = []
station_dist_2 = []

for index, row in fire_data.iterrows():
    

    lat = row['Lat']
    lng = row['Lng']
    dist_list = []

    for index1, row1 in dallas_coords_df.iterrows():
        lat1 = row1['Lat']
        lng1 = row1['Lng']
        dist = haversine(lat1,lng1,lat,lng,'mi')
        dist_list.append(dist)
    dist_list.sort()
    station_dist_1.append(dist_list[0])
    station_dist_2.append(dist_list[1])
fire_data['Dist to 1st Station (mi)'] = station_dist_1
fire_data['Dist to 2nd Station (mi)'] = station_dist_2
fire_data['Dist to 1st Station (mi)'] = fire_data['Dist to 1st Station (mi)'].round(2)
fire_data['Dist to 2nd Station (mi)'] = fire_data['Dist to 2nd Station (mi)'].round(2)

try:
    fire_data.to_csv('data/Dallas_coords_grid.csv',index=False)
    print('save successful')
except:
    print('save fail')


save successful


### Map of existing fires and distance to closest fire station

In [32]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, -96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = fire_data[['Lat','Lng']]

station_dist = fire_data['Dist to 1st Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=False, max_intensity=4, point_radius=.015)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(dallas_points)
# Add the layer to the map
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))